In [8]:
import polars as pl

# Read large CSV lazily
df = pl.scan_csv("./risk-pipeline/data/raw/DE1_0_2008_to_2010_Carrier_Claims_Sample_1B.csv")

# Convert to parquet (efficient, compressed, columnar)
df.sink_parquet("./risk-pipeline/data/parquet/carrier_2008.parquet")


ComputeError: could not parse `V0481` as dtype `i64` at column 'LINE_ICD9_DGNS_CD_3' (column number 132)

The current offset in the file is 1557 bytes.

You might want to try:
- increasing `infer_schema_length` (e.g. `infer_schema_length=10000`),
- specifying correct dtype with the `schema_overrides` argument
- setting `ignore_errors` to `True`,
- adding `V0481` to the `null_values` list.

Original error: ```remaining bytes non-empty```

In [7]:
import duckdb

# Connect to local DB file (persists across sessions)
con = duckdb.connect("synpuf.duckdb")

# Register parquet folder as a table
con.execute("""
    CREATE TABLE IF NOT EXISTS carrier_2008 AS 
    SELECT * FROM parquet_scan('./risk-pipeline/data/parquet/carrier_2008.parquet')
""")

# Example query: Top 10 beneficiaries by spend
df = con.execute("""
    SELECT DESYNPUF_ID
    FROM carrier_2008
""").fetchdf()

print(df)


             DESYNPUF_ID
0       00013D2EFD8E45D1
1       00016F745862898F
2       0001FDD721E223DC
3       00021CA6FF03E670
4       00024B3D2352D2D0
...                  ...
116347  FFFD7C464C94D136
116348  FFFE3ED9582AD46E
116349  FFFE7AB5D7F1725D
116350  FFFF29A2E1362059
116351  FFFF7C107A4E385A

[116352 rows x 1 columns]
